<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [7]:

// Базовый класс Inventory
public class Inventory
{
    // Основные атрибуты
    public string WarehouseId { get; set; }
    public string WarehouseName { get; set; }
    public double StorageCapacity { get; set; }
    
    // Новые атрибуты
    public string Location { get; set; }
    public string ContactPerson { get; set; }
    public DateTime CreatedDate { get; set; }
    public bool IsActive { get; set; }
    
    protected List<Item> items;
    protected double usedSpace;
    
    // События
    public event Action<string, string> ItemAdded;
    public event Action<string, string> ItemRemoved;
    public event Action<string, double> LowSpaceAlert;
    
    // Делегаты
    public delegate void InventoryOperationDelegate(string operation, string details);
    public InventoryOperationDelegate OperationLogger;
    
    // Коллекции
    public Dictionary<string, string> Tags { get; private set; }
    public Queue<string> OperationLog { get; private set; }
    public HashSet<string> RestrictedItems { get; private set; }

    public Inventory(string warehouseId, string warehouseName, double storageCapacity, string location, string contactPerson)
    {
        WarehouseId = warehouseId;
        WarehouseName = warehouseName;
        StorageCapacity = storageCapacity;
        Location = location;
        ContactPerson = contactPerson;
        CreatedDate = DateTime.Now;
        IsActive = true;
        
        items = new List<Item>();
        usedSpace = 0;
        
        Tags = new Dictionary<string, string>();
        OperationLog = new Queue<string>();
        RestrictedItems = new HashSet<string>();
        
        OperationLogger?.Invoke("Создание", $"Склад {WarehouseName} создан");
    }
    
    public virtual string GetStorageStatus()
    {
        double availableSpace = StorageCapacity - usedSpace;
        double usagePercentage = (usedSpace / StorageCapacity) * 100;
        
        return $"Склад '{WarehouseName}' (ID: {WarehouseId})\n" +
               $"Местоположение: {Location}\n" +
               $"Общая емкость: {StorageCapacity}\n" +
               $"Использовано: {usedSpace} ({usagePercentage:F1}%)\n" +
               $"Доступно: {availableSpace}";
    }
    
    public virtual void AddItem(Item item)
    {
        if (RestrictedItems.Contains(item.Name.ToLower()))
        {
            throw new InvalidOperationException($"Товар '{item.Name}' запрещен");
        }
        
        double requiredSpace = item.Volume * item.Quantity;
        if (usedSpace + requiredSpace > StorageCapacity)
        {
            throw new InvalidOperationException($"Недостаточно места. Требуется: {requiredSpace}, доступно: {StorageCapacity - usedSpace}");
        }
        
        items.Add(item);
        usedSpace += requiredSpace;
        
        // Вызов события
        ItemAdded?.Invoke(item.Name, WarehouseName);
        OperationLogger?.Invoke("Добавление", $"Товар {item.Name} добавлен");
        AddToOperationLog($"Добавлен: {item.Name}");
        
        // Проверка заполненности
        if ((usedSpace / StorageCapacity) * 100 > 80)
        {
            LowSpaceAlert?.Invoke(WarehouseName, (usedSpace / StorageCapacity) * 100);
        }
        
        Console.WriteLine($"Товар добавлен: {item}");
    }
    
    public virtual void RemoveItem(Item item)
    {
        if (items.Remove(item))
        {
            double freedSpace = item.Volume * item.Quantity;
            usedSpace -= freedSpace;
            
            // Вызов события - теперь в базовом классе
            ItemRemoved?.Invoke(item.Name, WarehouseName);
            OperationLogger?.Invoke("Удаление", $"Товар {item.Name} удален");
            AddToOperationLog($"Удален: {item.Name}");
            
            Console.WriteLine($"Товар удален: {item}");
        }
        else
        {
            Console.WriteLine($"Товар не найден: {item}");
        }
    }
    
    // Защищенный метод для вызова события из производных классов
    protected void OnItemRemoved(string itemName)
    {
        ItemRemoved?.Invoke(itemName, WarehouseName);
    }
    
    public virtual void DisplayAllItems()
    {
        Console.WriteLine($"\nТовары на складе '{WarehouseName}':");
        foreach (var item in items)
        {
            Console.WriteLine($"  - {item}");
        }
    }
    
    // Новые методы
    public void AddTag(string key, string value)
    {
        Tags[key] = value;
        AddToOperationLog($"Добавлен тег: {key}={value}");
    }
    
    public void AddRestrictedItem(string itemName)
    {
        RestrictedItems.Add(itemName.ToLower());
        AddToOperationLog($"Запрещен товар: {itemName}");
    }
    
    public void DisplayOperationLog()
    {
        Console.WriteLine($"\nЛог операций склада '{WarehouseName}':");
        foreach (var log in OperationLog)
        {
            Console.WriteLine($"  {log}");
        }
    }
    
    public Dictionary<string, int> GetItemStatistics()
    {
        return items.GroupBy(item => item.Category)
                   .ToDictionary(g => g.Key, g => g.Sum(item => item.Quantity));
    }
    
    protected void AddToOperationLog(string operation)
    {
        OperationLog.Enqueue($"{DateTime.Now:HH:mm:ss} - {operation}");
        if (OperationLog.Count > 50)
            OperationLog.Dequeue();
    }
}

// 1. Производный класс PersonalInventory
public class PersonalInventory : Inventory
{
    // Новые атрибуты
    public string OwnerName { get; set; }
    public string OwnerEmail { get; set; }
    public decimal MonthlyFee { get; set; }
    public List<string> AccessCodes { get; private set; }
    
    // События
    public event Action<string, DateTime> AccessGranted;
    
    // Коллекции
    public LinkedList<string> NotificationQueue { get; private set; }
    public Dictionary<DateTime, string> AccessLog { get; private set; }

    public PersonalInventory(string warehouseId, string warehouseName, double storageCapacity, 
                           string location, string contactPerson, string ownerName, string ownerEmail, decimal monthlyFee)
        : base(warehouseId, warehouseName, storageCapacity, location, contactPerson)
    {
        OwnerName = ownerName;
        OwnerEmail = ownerEmail;
        MonthlyFee = monthlyFee;
        
        AccessCodes = new List<string>();
        NotificationQueue = new LinkedList<string>();
        AccessLog = new Dictionary<DateTime, string>();
        
        GenerateAccessCodes();
    }
    
    public override string GetStorageStatus()
    {
        double availableSpace = StorageCapacity - usedSpace;
        double usagePercentage = (usedSpace / StorageCapacity) * 100;
        
        return $"Персональный склад '{WarehouseName}'\n" +
               $"Владелец: {OwnerName}\n" +
               $"Ежемесячная плата: {MonthlyFee:C}\n" +
               $"Общая емкость: {StorageCapacity}\n" +
               $"Использовано: {usedSpace} ({usagePercentage:F1}%)\n" +
               $"Доступно: {availableSpace}";
    }
    
    // Новые методы
    public void GenerateAccessCodes()
    {
        Random rand = new Random();
        for (int i = 0; i < 3; i++)
        {
            AccessCodes.Add(rand.Next(1000, 9999).ToString());
        }
        AddToOperationLog("Сгенерированы коды доступа");
    }
    
    public bool ValidateAccessCode(string code)
    {
        bool isValid = AccessCodes.Contains(code);
        AccessLog[DateTime.Now] = $"Код: {code} - {(isValid ? "Успех" : "Ошибка")}";
        
        if (isValid)
        {
            AccessGranted?.Invoke(OwnerName, DateTime.Now);
        }
        
        return isValid;
    }
    
    public void AddNotification(string message)
    {
        NotificationQueue.AddLast($"{DateTime.Now:HH:mm:ss} - {message}");
    }
    
    public void ProcessNotifications()
    {
        Console.WriteLine($"\nУведомления для {OwnerName}:");
        foreach (var notification in NotificationQueue)
        {
            Console.WriteLine($"  - {notification}");
        }
        NotificationQueue.Clear();
    }
    
    public decimal CalculateAnnualFee()
    {
        return MonthlyFee * 12;
    }
}

// 2. Производный класс GroupInventory
public class GroupInventory : Inventory
{
    // Новые атрибуты
    public string ProductGroup { get; set; }
    public string CategoryManager { get; set; }
    public int MaxItemsPerGroup { get; set; }
    
    // Коллекции
    public Dictionary<string, List<Item>> GroupCategories { get; private set; }
    public Queue<Item> ApprovalQueue { get; private set; }
    public HashSet<string> ApprovedSuppliers { get; private set; }
    
    // События
    public event Action<string> ApprovalRequired;

    public GroupInventory(string warehouseId, string warehouseName, double storageCapacity,
                         string location, string contactPerson, string productGroup, string categoryManager)
        : base(warehouseId, warehouseName, storageCapacity, location, contactPerson)
    {
        ProductGroup = productGroup;
        CategoryManager = categoryManager;
        MaxItemsPerGroup = 100;
        
        GroupCategories = new Dictionary<string, List<Item>>();
        ApprovalQueue = new Queue<Item>();
        ApprovedSuppliers = new HashSet<string>();
    }
    
    public override void AddItem(Item item)
    {
        if (items.Count >= MaxItemsPerGroup)
        {
            throw new InvalidOperationException($"Достигнут лимит товаров в группе. Максимум: {MaxItemsPerGroup}");
        }
        
        if (!ApprovedSuppliers.Contains(item.Supplier))
        {
            ApprovalQueue.Enqueue(item);
            ApprovalRequired?.Invoke(item.Name);
            Console.WriteLine($"Товар '{item.Name}' добавлен в очередь на одобрение");
            return;
        }
        
        base.AddItem(item);
        
        // Добавляем в категорию
        if (!GroupCategories.ContainsKey(item.Category))
        {
            GroupCategories[item.Category] = new List<Item>();
        }
        GroupCategories[item.Category].Add(item);
        
        Console.WriteLine($"Товар добавлен в группу '{ProductGroup}': {item}");
    }
    
    public override string GetStorageStatus()
    {
        return $"Групповой склад '{WarehouseName}'\n" +
               $"Группа товаров: {ProductGroup}\n" +
               $"Менеджер: {CategoryManager}\n" +
               $"Категории: {GroupCategories.Count}\n" +
               $"Ожидают одобрения: {ApprovalQueue.Count}\n" +
               $"Общая емкость: {StorageCapacity}\n" +
               $"Использовано: {usedSpace} ({GetUtilizationPercentage():F1}%)\n" +
               $"Доступно: {StorageCapacity - usedSpace}";
    }
    
    // Новые методы
    public void AddApprovedSupplier(string supplier)
    {
        ApprovedSuppliers.Add(supplier);
        AddToOperationLog($"Одобрен поставщик: {supplier}");
    }
    
    public void ProcessApprovalQueue()
    {
        Console.WriteLine($"\nОбработка очереди одобрения:");
        while (ApprovalQueue.Count > 0)
        {
            var item = ApprovalQueue.Dequeue();
            // Теперь добавляем товар через базовый метод
            base.AddItem(item);
            Console.WriteLine($"  Одобрен и добавлен: {item.Name}");
        }
    }
    
    public void DisplayCategoryStatistics()
    {
        Console.WriteLine($"\nСтатистика по категориям:");
        foreach (var category in GroupCategories)
        {
            Console.WriteLine($"  {category.Key}: {category.Value.Count} товаров");
        }
    }
    
    private double GetUtilizationPercentage()
    {
        return (usedSpace / StorageCapacity) * 100;
    }
}

// 3. Производный класс AutomatedInventory
public class AutomatedInventory : Inventory
{
    // Новые атрибуты
    public int AutomationLevel { get; set; }
    public string SoftwareVersion { get; set; }
    public bool IsOnline { get; set; }
    
    // Коллекции
    public Queue<string> TaskQueue { get; private set; }
    public Dictionary<string, int> PerformanceMetrics { get; private set; }

    public AutomatedInventory(string warehouseId, string warehouseName, double storageCapacity,
                            string location, string contactPerson, int automationLevel, string softwareVersion)
        : base(warehouseId, warehouseName, storageCapacity, location, contactPerson)
    {
        AutomationLevel = automationLevel;
        SoftwareVersion = softwareVersion;
        IsOnline = true;
        
        TaskQueue = new Queue<string>();
        PerformanceMetrics = new Dictionary<string, int>
        {
            ["items_processed"] = 0,
            ["errors_count"] = 0
        };
    }
    
    public override void RemoveItem(Item item)
    {
        if (!IsOnline)
        {
            throw new InvalidOperationException("Склад офлайн");
        }
        
        if (items.Remove(item))
        {
            double freedSpace = item.Volume * item.Quantity;
            usedSpace -= freedSpace;
            PerformanceMetrics["items_processed"]++;
            
            Console.WriteLine($"Товар удален автоматически (уровень {AutomationLevel}): {item}");
            
            // Используем защищенный метод базового класса для вызова события
            OnItemRemoved(item.Name);
            AddToOperationLog($"Автоудаление: {item.Name}");
        }
    }
    
    public override string GetStorageStatus()
    {
        return $"Автоматизированный склад '{WarehouseName}'\n" +
               $"Уровень автоматизации: {AutomationLevel}/5\n" +
               $"Версия ПО: {SoftwareVersion}\n" +
               $"Статус: {(IsOnline ? "Онлайн" : "Офлайн")}\n" +
               $"Задачи в очереди: {TaskQueue.Count}\n" +
               $"Общая емкость: {StorageCapacity}\n" +
               $"Использовано: {usedSpace} ({GetUtilizationPercentage():F1}%)\n" +
               $"Доступно: {StorageCapacity - usedSpace}";
    }
    
    // Новые методы
    public void AddTask(string task)
    {
        TaskQueue.Enqueue(task);
        AddToOperationLog($"Добавлена задача: {task}");
    }
    
    public void ProcessTasks()
    {
        Console.WriteLine($"\nОбработка задач:");
        while (TaskQueue.Count > 0)
        {
            var task = TaskQueue.Dequeue();
            Console.WriteLine($"  Выполнено: {task}");
        }
    }
    
    public void PerformAutomatedCheck()
    {
        Console.WriteLine($"\nАвтоматическая проверка склада:");
        Console.WriteLine($"Товаров: {items.Count}");
        Console.WriteLine($"Заполненность: {GetUtilizationPercentage():F1}%");
        Console.WriteLine($"Обработано товаров: {PerformanceMetrics["items_processed"]}");
    }
    
    public void ToggleOnlineStatus()
    {
        IsOnline = !IsOnline;
        AddToOperationLog($"Статус: {(IsOnline ? "Онлайн" : "Офлайн")}");
    }
    
    private double GetUtilizationPercentage()
    {
        return (usedSpace / StorageCapacity) * 100;
    }
}

// Класс Item с новыми свойствами
public class Item
{
    public string Name { get; set; }
    public double Volume { get; set; }
    public int Quantity { get; set; }
    public string Category { get; set; }
    public string Supplier { get; set; }
    
    public Item(string name, double volume, int quantity)
    {
        Name = name;
        Volume = volume;
        Quantity = quantity;
        Category = "Общее";
        Supplier = "Неизвестно";
    }
    
    public override string ToString()
    {
        return $"{Name} (Объем: {Volume}, Количество: {Quantity}, Категория: {Category})";
    }
}

// Демонстрация
        Console.WriteLine("=== СИСТЕМА УПРАВЛЕНИЯ СКЛАДАМИ ===\n");
        
        // Создание складов
        var personal = new PersonalInventory("WH001", "Мой склад", 500, "Москва", 
                                           "Иван Иванов", "Петр Сидоров", "peter@mail.com", 5000m);
        
        var group = new GroupInventory("WH002", "Электроника", 2000, "СПб", 
                                     "Анна Петрова", "Электроника", "Мария Козлова");
        
        var auto = new AutomatedInventory("WH003", "Авто-склад", 3000, "Казань",
                                        "Сергей Волков", 4, "2.1.0");

        // Подписка на события
        personal.ItemAdded += (item, warehouse) => 
            Console.WriteLine($"[СОБЫТИЕ] Товар {item} добавлен на склад {warehouse}");
        
        personal.LowSpaceAlert += (warehouse, percent) => 
            Console.WriteLine($"[ПРЕДУПРЕЖДЕНИЕ] Склад {warehouse} заполнен на {percent:F1}%");
        
        personal.AccessGranted += (owner, time) => 
            Console.WriteLine($"[ДОСТУП] {owner} получил доступ в {time:HH:mm:ss}");
        
        group.ApprovalRequired += (item) => 
            Console.WriteLine($"[ОДОБРЕНИЕ] Требуется одобрение для {item}");

        auto.ItemRemoved += (item, warehouse) =>
            Console.WriteLine($"[АВТОМАТИЗАЦИЯ] Товар {item} автоматически удален с {warehouse}");

        // Делегат для логирования
        Inventory.InventoryOperationDelegate logger = (op, details) => 
            Console.WriteLine($"[ЛОГ] {op}: {details}");
        
        personal.OperationLogger = logger;
        group.OperationLogger = logger;
        auto.OperationLogger = logger;

        // Тестовые товары
        var laptop = new Item("Ноутбук", 5.0, 10) { Category = "Электроника", Supplier = "TechCorp" };
        var phone = new Item("Смартфон", 1.0, 50) { Category = "Электроника", Supplier = "PhoneInc" };
        var book = new Item("Книга", 0.5, 100) { Category = "Офис", Supplier = "BookStore" };

        // Демонстрация
        Console.WriteLine("1. ПЕРСОНАЛЬНЫЙ СКЛАД:");
        personal.AddItem(laptop);
        personal.AddItem(book);
        personal.ValidateAccessCode("1234");
        personal.ValidateAccessCode(personal.AccessCodes[0]);
        personal.AddNotification("Техобслуживание запланировано");
        personal.ProcessNotifications();
        Console.WriteLine(personal.GetStorageStatus());
        
        Console.WriteLine("\n2. ГРУППОВОЙ СКЛАД:");
        group.AddApprovedSupplier("TechCorp");
        group.AddItem(laptop);
        group.AddItem(phone); // Будет в очереди одобрения
        group.ProcessApprovalQueue();
        group.DisplayCategoryStatistics();
        Console.WriteLine(group.GetStorageStatus());
        
        Console.WriteLine("\n3. АВТОМАТИЗИРОВАННЫЙ СКЛАД:");
        auto.AddItem(laptop);
        auto.AddItem(phone);
        auto.RemoveItem(phone); // Теперь работает корректно
        auto.AddTask("Инвентаризация");
        auto.AddTask("Оптимизация");
        auto.ProcessTasks();
        auto.PerformAutomatedCheck();
        Console.WriteLine(auto.GetStorageStatus());
        
        Console.WriteLine("\n4. ЛОГИ ОПЕРАЦИЙ:");
        personal.DisplayOperationLog();
        group.DisplayOperationLog();
        
        Console.WriteLine("\n5. СТАТИСТИКА:");
        var stats = personal.GetItemStatistics();
        foreach (var stat in stats)
        {
            Console.WriteLine($"  {stat.Key}: {stat.Value} шт.");
        }


=== СИСТЕМА УПРАВЛЕНИЯ СКЛАДАМИ ===

1. ПЕРСОНАЛЬНЫЙ СКЛАД:
[СОБЫТИЕ] Товар Ноутбук добавлен на склад Мой склад
[ЛОГ] Добавление: Товар Ноутбук добавлен
Товар добавлен: Ноутбук (Объем: 5, Количество: 10, Категория: Электроника)
[СОБЫТИЕ] Товар Книга добавлен на склад Мой склад
[ЛОГ] Добавление: Товар Книга добавлен
Товар добавлен: Книга (Объем: 0.5, Количество: 100, Категория: Офис)
[ДОСТУП] Петр Сидоров получил доступ в 15:47:37

Уведомления для Петр Сидоров:
  - 15:47:37 - Техобслуживание запланировано
Персональный склад 'Мой склад'
Владелец: Петр Сидоров
Ежемесячная плата: ¤5,000.00
Общая емкость: 500
Использовано: 100 (20.0%)
Доступно: 400

2. ГРУППОВОЙ СКЛАД:
[ЛОГ] Добавление: Товар Ноутбук добавлен
Товар добавлен: Ноутбук (Объем: 5, Количество: 10, Категория: Электроника)
Товар добавлен в группу 'Электроника': Ноутбук (Объем: 5, Количество: 10, Категория: Электроника)
[ОДОБРЕНИЕ] Требуется одобрение для Смартфон
Товар 'Смартфон' добавлен в очередь на одобрение

Обработка очереди 